In [51]:
import torch
from networks import FullyConnected, SPU, Normalization
import os
import time

DEVICE = 'cpu'
INPUT_SIZE = 28

In [50]:
class Args:
    net: str
    spec: str

    def __init__(self, net: str, spec: str):
        self.net = net
        self.spec = spec

In [28]:
def analyze(net, inputs, eps, true_label):
    start_time = time.time()

    # Compute lower and upper bounds for all points
    inputs_lx = inputs.detach() - eps * 1
    inputs_ux = inputs.detach() + eps * 1

    for layer in net.layers:
        if type(layer) is SPU:
            continue
        
        elif type(layer) is Normalization:
            continue
        
        elif type(layer) is torch.nn.Flatten:
            continue
    

    # ???

    # Compare around 
    end_time = time.time()
    
    return 0

    # return (true_label_lx > labels_ux).all()

In [52]:
args = Args(
    net="net0_fc1",
    spec="../test_cases/net0_fc1/example_img1_0.05000.txt"
)

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])

if args.net.endswith('fc1'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [50, 10]).to(DEVICE)

elif args.net.endswith('fc2'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc3'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 10]).to(DEVICE)

elif args.net.endswith('fc4'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc5'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [
                            100, 100, 100, 100, 10]).to(DEVICE)
else:
    assert False

net.load_state_dict(torch.load('../mnist_nets/%s.pt' %
                    args.net, map_location=torch.device(DEVICE)))

inputs = torch.FloatTensor(pixel_values).view(
    1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)

outs = net(inputs)
pred_label = outs.max(dim=1)[1].item()
assert pred_label == true_label

print(f'Network: {net}')
print(f'Inputs shape: {inputs.shape}')
print(f'Epsilon: {eps}')
print(f'True label: {true_label}')

print()

if analyze(net, inputs, eps, true_label):
    print('verified'.capitalize())
else:
    print('not verified'.capitalize())


Network: FullyConnected(
  (layers): Sequential(
    (0): Normalization()
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=784, out_features=50, bias=True)
    (3): SPU()
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
)
Inputs shape: torch.Size([1, 1, 28, 28])
Epsilon: 0.05
True label: 3

Not verified
